In [ ]:
#Written by Ranyi Zhang
!pip install diffusers transformers accelerate safetensors
from diffusers import StableDiffusionXLPipeline
import torch

pipe = StableDiffusionXLPipeline.from_pretrained(
    "stabilityai/stable-diffusion-xl-base-1.0",
    torch_dtype=torch.float16,
    variant="fp16",
    use_safetensors=True
).to("cuda")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

In [4]:
!pip install flask flask-ngrok


In [5]:
from google.colab import userdata
!pip install flask
!wget -q -O ngrok.zip https://bin.equinox.io/c/bNyj1mQVY4c/ngrok-stable-linux-amd64.zip
!unzip -o ngrok.zip
!chmod +x ngrok
!./ngrok config add-authtoken userdata.get('NGROK_AUTHTOKEN')


Archive:  ngrok.zip
  inflating: ngrok                   
Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


In [6]:
from flask import Flask, request, jsonify
from threading import Thread
import io, base64

app = Flask(__name__)

@app.route('/generate-avatar', methods=['POST'])
def generate_avatar():
    try:
        data = request.get_json()
        prompt = data.get("prompt", "A fantasy anime avatar")

        print(f"🧠 Received prompt: {prompt}", flush=True)

        image = pipe(prompt).images[0]
        buffered = io.BytesIO()
        image.save(buffered, format="PNG")
        img_base64 = base64.b64encode(buffered.getvalue()).decode("utf-8")

        return jsonify({"status": "success", "imageBase64": img_base64})

    except Exception as e:
        error_msg = str(e)
        # print to stdout
        print("❌ Error occurred:", error_msg, flush=True)
        # save to a visible file
        with open("error.log", "a") as f:
            f.write(error_msg + "\n")
        return jsonify({"status": "error", "message": error_msg}), 500


In [9]:
def run_flask():
    app.run(host='0.0.0.0', port=5000)

flask_thread = Thread(target=run_flask)
flask_thread.start()


 * Serving Flask app '__main__'
 * Debug mode: off


In [10]:
import subprocess
import time
import requests

ngrok = subprocess.Popen(["./ngrok", "http", "5000"])
time.sleep(3)

res = requests.get("http://localhost:4040/api/tunnels")
public_url = res.json()["tunnels"][0]["public_url"]
print("✅ Flask is exposed at:", public_url)


✅ Flask is exposed at: https://d536-104-199-118-106.ngrok-free.app


In [11]:
!ls -l

total 36364
-rwxr-xr-x 1 root root 27762872 Apr 10 19:59 ngrok
-rw-r--r-- 1 root root  9462138 May  6 05:15 ngrok.zip
drwxr-xr-x 1 root root     4096 May  2 13:33 sample_data


In [22]:
#!fuser -k 5000/tcp
!whoami

root
